# Driving ParaView from a Jupyter Notebook

Start the ParaView server to allow connections from a `pvpython` client and a ParaView GUI client
```bash
pvserver --multi-clients &
/Applications/paraview4.3.1.app/Contents/MacOS/paraview --server-url=cs://localhost:11111
```

Start ParaView gui and connect to the server now! Close the plugin window if it appears (this script will load the plugins) and open the Collaboration Panel.

In [1]:
%load_ext ipythonPexpect

%ipythonPexpect? for help


In [2]:
%pexpect_spawn_pvpython

Opened connection to pvpython
Python 2.7.6 (default, Sep  9 2014, 15:04:36) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.39)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 

In [3]:
%pexpect_lock

To return to IPython, issue %pexpect_unlock


In [4]:
from paraview.simple import *
from paraview import servermanager
from paraview.collaboration import *

paraview version 4.3.1


Connect to the remove pvserver

In [5]:
rc = Connect("localhost")
rc

Connection (cs://localhost:11111) [2]


Change our identifier and make the ParaView Gui follow this user (me).

In [6]:
cm = rc.Session.GetCollaborationManager()
cm.UpdateUserInformations()
myUserId = cm.GetUserId()
cm.SetUserLabel(myUserId, "JupyterNotebook")
cm.PromoteToMaster(myUserId)
cm.FollowUser(myUserId)

Load in the GeantToVTK Plugin on the remote server.

In [7]:
LoadPlugin("/Users/lyon/ParaView/GeantToVTK-1.2/GeantToVTKPlugin.xml", True, globals())

Load in the Straw test beam geometry. Note that we'll remove some structures.

In [8]:
geom = HeprepXMLGeometryView(FileName='/Users/lyon/Development/g-2/temp/forJoePrice/price.heprep.zip')
geom.InvertRegularExpression = 1
geom.RegularExpression = 'ModuleChamberLV$|vacuumModuleLV'

Display it

In [9]:
rv = GetActiveViewOrCreate("RenderView")

In [10]:
geomDisplay = Show(geom, rv)

Change the colors

In [11]:
ColorBy(geomDisplay, ('FIELD', 'Color'))
geomDisplay.MapScalars = 0

Let's move the camera to something reasonable

In [12]:
rv.CameraPosition = [-821.9220570132178, -837.5306099552349, 157.21331804780507]
rv.CameraFocalPoint = [-48.747505187988246, -17.9000015258789, 8.424669622740802e-16]
rv.CameraViewUp = [0.3046624436996831, -0.10583435104923601, 0.9465621403462767]
rv.CameraParallelScale = 254.45250809295567

Now load in an event. 

In [13]:
event = HeprepXMLeventviewer(FileName='/Users/lyon/Development/g-2/temp/forJoePrice/price.heprep.zip')

We want the first event (first time step in the animation)

In [14]:
aniScene = GetAnimationScene()
aniScene.UpdateAnimationUsingDataTimeSteps()
aniScene.PlayMode = 'Snap To TimeSteps'

True


Display it

In [15]:
eventDisplay = Show(event, rv)

Change the name (must do this at the end of processing to avoid losing pipeline entries - I imagine some code somewhere looks for the original names in the Pipeline).

In [16]:
viewLayout = GetLayout()
SaveScreenshot('/Users/lyon/dataAnalysisPlatform/Documents/Paraview/strawTB-event1.png', 
               layout=viewLayout, magnification=1, quality=100)

True


Here's the snapshot
![Image](strawTB-event1.png)
Note the Geant generated axes thing in the lower left cornel.

In [26]:
servermanager.SaveState("strawTB-event1.pvsm")   # SaveState from simple doesn't seem to work

Ask the event reader for the timesteps

In [28]:
steps = event.TimestepValues ; steps

[1.0, 2.0, 3.0, 4.0, 5.0]


In [29]:
aniScene.AnimationTime ; rv.ViewTime

1.0
1.0


Change the event

In [30]:
aniScene.AnimationTime = steps[4]

In [31]:
aniScene.AnimationTime ; rv.ViewTime

5.0
5.0


In [32]:
viewLayout = GetLayout()
SaveScreenshot('/Users/lyon/dataAnalysisPlatform/Documents/Paraview/strawTB-event5.png', 
               layout=viewLayout, magnification=1, quality=100)

True


Here is event 5
![event5](strawTB-event5.png)

In [33]:
servermanager.SaveState("strawTB-event5.pvsm")   # SaveState from simple doesn't seem to work

In [34]:
RenameSource('Geometry', geom)

In [35]:
RenameSource('Events', event)

In [36]:
Disconnect()

In [37]:
%pexpect_unlock

Notebook will use IPython


In [38]:
%pexpect_close

Closed connection to pvpython
